In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import openreview
from collections import Counter, defaultdict as dd
import numpy as np
import pandas as pd
import qgrid
from pprint import pprint
import datetime
import pickle
import re 
import time
# import jellyfish
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict as ddict 

sns.set(rc={'figure.figsize':(20,8.27)}, font_scale=2)

/home/s.ivanov/miniconda3/envs/deepr_python/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [21]:
review_url

'ICLR.cc/2020/Conference/Paper*/-/Official_Review'

In [20]:
!jupyter nbextension enable --py --sys-prefix qgrid
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

Enabling notebook extension qgrid/extension...
      - Validating: OK
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [13]:
URL = 'ICLR.cc/2020/Conference/-/Blind_Submission'

def all_papers():
    client = openreview.Client(baseurl='https://openreview.net')
    paper_iterator = openreview.tools.iterget_notes(client, invitation=URL)
    return list(paper_iterator)

In [14]:
s = all_papers()

In [31]:
pprint(s[0].__dict__)

{'cdate': 1569439736148,
 'content': {'TL;DR': 'We augment model-free policy learning with a '
                      'sequence-level surrogate reward functions and '
                      'count-based visitation bonus and demonstrate '
                      'effectiveness in the large batch, low-round regime seen '
                      'in designing DNA and protein sequences.',
             '_bibtex': '@inproceedings{\n'
                        'Angermueller2020Model-based,\n'
                        'title={Model-based reinforcement learning for '
                        'biological sequence design},\n'
                        'author={Christof Angermueller and David Dohan and '
                        'David Belanger and Ramya Deshpande and Kevin Murphy '
                        'and Lucy Colwell},\n'
                        'booktitle={International Conference on Learning '
                        'Representations},\n'
                        'year={2020},\n'
                      

In [61]:
review_URL = 'ICLR.cc/2020/Conference/Paper.*/-/Official_Review'
def ratings_and_confidence():
    client = openreview.Client(baseurl='https://openreview.net')
    review_iterator = openreview.tools.iterget_notes(client, invitation=review_URL)
    ratings = dd(list)
    confidence = dd(list)
    for review in review_iterator:
        ratings[review.forum].append(int(re.findall(pattern='\d+', string=review.content['rating'])[0]))
#         confidence[review.forum].append(int(re.findall(pattern='\d+', string=review.content['confidence'])[0]))
    return ratings, confidence

In [62]:
papers = all_papers()
ratings, confidence = ratings_and_confidence()

In [63]:
def add_ratings(papers, ratings, confidence):
    for ix in range(len(papers)):
        forum = papers[ix].forum
        papers[ix].ratings = ratings.get(forum, [])
        papers[ix].confidence = confidence.get(forum, [])

In [64]:
add_ratings(papers, ratings, dict())

In [66]:
pprint(papers[0].__dict__)

{'cdate': 1569439786153,
 'confidence': [],
 'content': {'TL;DR': 'We introduce a class of n-player games suited to '
                      'gradient-based methods.',
             '_bibtex': '@inproceedings{\n'
                        'Balduzzi2020Smooth,\n'
                        'title={Smooth markets: A basic mechanism for '
                        'organizing gradient-based learners},\n'
                        'author={David Balduzzi and Wojciech M. Czarnecki and '
                        'Edward Hughes and Joel Leibo and Ian Gemp and Tom '
                        'Anthony and Georgios Piliouras and Thore Graepel},\n'
                        'booktitle={International Conference on Learning '
                        'Representations},\n'
                        'year={2020},\n'
                        'url={https://openreview.net/forum?id=B1xMEerYvB}\n'
                        '}',
             'abstract': 'With the success of modern machine learning, it is '
                     

In [41]:
inv = openreview.tools.get_submission_invitations(client)
print(*inv,sep="\n")

AKBC.ws/2020/Conference/-/Submission
MIDL.io/2020/Conference/-/Submission
MSMARCO.org/2020/Task/-/Submission
ICLR.cc/2020/Workshop/DeepDiffEq/-/Submission
TimeMachine.eu/RFC/-/Submission
acm.org/CHI/2020/Workshop/AI4HCI/-/Submission
ICLR.cc/2020/Workshop/AI4AH/-/Submission
msrconf.org/MSR/2020/Technical_Track/-/Submission
conceptuccino.uni-osnabrueck.de/CARLA/2020/Workshop/-/Submission
ICLR.cc/2020/Conference/-/Desk_Rejected_Submission
ICLR.cc/2020/Conference/-/Withdrawn_Submission
sydney.edu.au/EDSC/2020/Conference/-/Submission
ICML.cc/2019/Workshop/AMTL/-/Blind_Submission
icaps-conference.org/ICAPS/2019/Workshop/XAIP/-/Blind_Submission
icaps-conference.org/ICAPS/2019/Workshop/WIPC/-/Blind_Submission
icaps-conference.org/ICAPS/2019/Workshop/SPARK/-/Blind_Submission
icaps-conference.org/ICAPS/2019/Workshop/Hierarchical_Planning/-/Blind_Submission
learningtheory.org/COLT/2019/Conference/-/Withdrawn_Submission
icaps-conference.org/ICAPS/2019/Workshop/HSDIP/-/Blind_Submission
ICLR.cc/2019

In [ ]:
paper123_reviews = client.get_notes( invitation='ICLR.cc/2020/Conference/Paper1/-/Official_Review')

In [8]:
paper123_decision = client.get_notes(invitation='ICLR.cc/2020/Conference/Paper1/-/Decision')
paper123_decision[0].__dict__

paper_URL = 'ICLR.cc/2020/Conference/-/Blind_Submission'
decision_URL = 'ICLR.cc/2020/Conference/Paper.*/-/Decision'
review_URL = 'ICLR.cc/2020/Conference/Paper*/-/Official_Review'

def get_notes(url):
    client = openreview.Client(baseurl='https://openreview.net')
    iterator = openreview.tools.iterget_notes(client, invitation=url)
    notes = ddict(list)
    for i in iterator:
        notes[i.forum].append(i)
    return notes

papers = get_notes(paper_URL)
decisions = get_notes(decision_URL)
reviews = get_notes(review_URL)

def rate(review):
    return re.findall('\d+', review.content['rating'])[0]

def merge_info(papers, decisions, reviews):
    seq = []
    for forum in papers:
        paper = papers[forum]
        decision = decisions[forum]
        review = reviews[forum]
        
        assert len(paper) == 1, 'there are more than 1 paper'
        assert len(decision) == 1, 'there are more than 1 decision'
        p = paper[0]
        title = p.content['title']
        url = "https://openreview.net/forum?id=" + forum
        d = decision[0]
        final_decision = d.content['decision']
        rates = [rate(r) for r in review]
        seq.append([title, url, final_decision, rates])
    df = pd.DataFrame(seq)
    df.columns = ['title', 'url', 'decision', 'ratings']
    return df
    
        

iclr_results = merge_info(papers, decisions, reviews)

In [ ]:
iclr_results

In [148]:
!jupyter nbextension enable --py --sys-prefix qgrid

Enabling notebook extension qgrid/extension...
      - Validating: OK


In [146]:
import qgrid
qgrid.show_grid(iclr_results.iloc[:10])

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [19]:
qgrid.show_grid(iclr_results.iloc[:10])

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [132]:
for r in reviews[id]:
    print(rate(r))

6
6
6


In [89]:
len(papers), len(decisions), len(reviews)

(2213, 2213, 7775)

In [60]:
paper123_reviews = client.get_notes( invitation='ICLR.cc/2020/Conference/Paper1/-/Official_Review')
for rew in paper123_reviews:
    pprint(rew.__dict__)

{'cdate': 1571624300614,
 'content': {'experience_assessment': 'I have published in this field for '
                                      'several years.',
             'rating': '3: Weak Reject',
             'review': '\n'
                       'In this paper, the authors made a study on large-batch '
                       'training for the BERT, and successfully trained a BERT '
                       'model in 76 minutes. The results look quite exciting, '
                       'however, after looking into the details of the paper, '
                       'I would say that this is just a kind of RED AI – the '
                       'results were mostly achieved by putting together a '
                       'huge number of TPUs, without necessary technical '
                       'innovation and fundamental contributions.\n'
                       '\n'
                       '1)\tThe work used 1024 TPUs to achieve 76-min '
                       'training. If we compare this

In [42]:
review_iterator = openreview.tools.iterget_notes(client, invitation='ICLR.cc/2019/Conference/-/Paper.*/Official_Review')
next(review_iterator)

Note(id = 'r1g3yc1c2X',original = None,number = 3,cdate = 1541171683697,tcdate = 1541171683697,tmdate = 1545388503557,ddate = None,content = {'title': 'interesting approach to generalization, but are the guarantees relevant?', 'review': 'The paper studies generalization through a general empirical risk minimization procedure with Lipschitz regularization.\nGeneralization is measured through distance of the empirical minimizer function to a true labeling function u_0, or to the minimizer of the expected regularized loss.\n\nThe approach of studying generalization through the lens of Lipschitz stability over the data is interesting,\nand the study directly considers minimizers of regularized optimization objectives, which is different than\nrecent generalization bounds which often provide guarantees on a given network regardless of how it was trained.\n\nHowever, various aspects of the setup seem quite disconnected from practice in the context of deep neural networks,\nand the obtained g

In [ ]:
invitations = openreview.tools.iterget_invitations(client)
for inv in invitations:
    if '2020' in inv.id and 'iclr' in inv.id.lower():
        print(inv.id)

In [54]:
paper123_reviews = client.get_notes(
        invitation='ICLR.cc/2020/Conference/-/Paper123/Submission')
paper123_reviews

[]

In [ ]:
review_URL = 'ICLR.cc/2019/Conference/-/Paper*/Submission'
client = openreview.Client(baseurl='https://openreview.net')
review_iterator = openreview.tools.iterget_notes(client, invitation=review_URL)
next(review_iterator)

In [ ]:
def get_keywords(papers):
    return list(set([word.strip().lower() for p in papers for word in p.content['keywords']]))

def get_similarity(keywords):
    sim = dict()
    N = len(keywords)
    for i in range(N-1):
        for j in range(i+1, N):
            topic1 = keywords[i]
            topic2 = keywords[j]
            dist = len(set(topic1).intersection(set(topic2)))/len(set(max([topic1, topic2], key=lambda v: len(v))))
            if topic1 != topic2 and dist >= 0.70:
                dist = jellyfish.levenshtein_distance(topic1, topic2)
                sim[topic1, topic2] = dist
    return sim

def get_top_matches(sim):
    matches = dd(list)
    for pair in sim:
        if sim[pair] == 1:
            matches[pair[0]].append(pair[1])
            matches[pair[1]].append(pair[0])
    return matches    

In [ ]:
keywords = get_keywords(papers)
start = time.time()
sim = get_similarity(keywords) # this may take several minutes 
finish = time.time()
print('Similarity computed in {:.2f} sec'.format(finish - start))

In [ ]:
matches = get_top_matches(sim)

In [ ]:
def get_clusters(matches):
    clusters = dict()
    for topic in matches:
        values = matches[topic]
        clusters[topic] = topic
        for v in values:
            if len(matches.get(v, [])) > len(values) or \
            (len(matches.get(v, [])) == len(values) and topic > v):
                clusters[topic] = v
    # second pass 
    main = set(clusters.items())
    for topic in clusters:
        if topic not in main:
            clusters[topic] = clusters[clusters[topic]]
    return clusters

In [ ]:
clusters = get_clusters(matches)
clusters.update({topic: 'generative adversarial networks' for topic in clusters if clusters[topic] == 'gan'})

In [ ]:
def get_df(papers, clusters):
    rows = []
    for paper in papers:
        topic = paper.content['keywords'][0] if len(paper.content['keywords']) else ''
        cluster = clusters.get(topic.lower(), topic.lower() if topic else None)
        row = dict(tldr = paper.content.get('TL;DR', ''),
                  ratings = getattr(paper, 'ratings', []),
                  confidence = getattr(paper, 'confidence', []),
                  title = paper.content['title'],
                  avg_rating = np.mean(getattr(paper, 'ratings', [])),
                  avg_confidence = np.mean(getattr(paper, 'confidence', [])),
                  topic = cluster,
                  url = "https://openreview.net/forum?id=" + paper.forum
                  )
        rows.append(row)
    df = pd.DataFrame(rows)
    df.index.name = 'Index'
    df = df.iloc[:, [4,1,0,6,5,3,2,7]]
    return df 

In [ ]:
df = get_df(papers, clusters)

In [ ]:
all_ratings = df['ratings'].agg(sum)
all_confidence = df['confidence'].agg(sum)
avg_rating = np.mean(all_ratings)
avg_confidence = np.mean(all_confidence)
import datetime
print('Statistics for', datetime.datetime.now().date())
print('Average rating: {:.1f} +- {:.1f}'.format(avg_rating, np.std(all_ratings)))
print('Average confidence: {:.1f}'.format(avg_confidence))
print('Total number of reviews:', len(all_ratings))
print('Total number of papers:', df.shape[0])

In [ ]:
Counter(all_ratings)

In [ ]:
c = sorted(Counter(all_ratings).items())
freq = pd.DataFrame(c, columns = ['score', 'number'])
bars = sns.barplot(x='score', y='number', data=freq)
# bars.set_title('Frequency of Review Score')
bars.set_ylabel('Frequency')
bars.set_xlabel('Score')
# xlabels = bars.set_xticklabels(bars.get_xticklabels(), rotation=90)
plt.show()

In [ ]:
def plot_heatmap(df):
    all_ratings = df['ratings'].agg(sum)
    all_confidence = df['confidence'].agg(sum)
    N, M = len(set(all_confidence)), len(set(all_ratings))
    heat = np.zeros((N, M))
    c = Counter(zip(all_confidence, all_ratings))
    for key in c:
        row, col = key
        heat[row-1, col-1] = c[key]
    normalized_heat = heat/np.linalg.norm(heat, axis=0)
    
    ax = sns.heatmap(normalized_heat, xticklabels=list(range(1,11)), yticklabels=list(range(1,6)), linewidths=.5)
#     ax.set_title('Confidence in Reviews. \nNormalized for each review score.')
    ax.set_xlabel('Score')
    ax.set_ylabel('Confidence')
    plt.show()
    
def plot_frequencies(df):
    topics = Counter(df.topic).most_common(50)
    topics = list(filter(lambda v: v[0], topics))
    freq = pd.DataFrame(topics, columns = ['topic', 'occurences'])
    bars = sns.barplot(x='topic', y='occurences', data=freq)
#     bars.set_title('Frequency of Popular Topics.')
    bars.set_xlabel('Topics')
    bars.set_ylabel('Submissions')
    xlabels = bars.set_xticklabels(bars.get_xticklabels(), rotation=90)
    plt.show()

In [ ]:
plot_heatmap(df)

In [ ]:
plot_frequencies(df)

In [ ]:
col_defs = {
    name: {
        'width': 50
    } for name in ['Index', 'confidence', 'ratings']
}
col_defs['title'] = {'width': 500}
col_defs['avg_rating'] = {'width': 100}
col_defs['tldr'] = {'width': 1000}
col_defs['url'] = {'width': 300}
col_defs['topic'] = {'width': 200}
grid_options={'forceFitColumns': False, 'defaultColumnWidth': 100}
grid = qgrid.show_grid(df, column_definitions=col_defs, grid_options = grid_options, precision=1)

In [ ]:
grid